In [ ]:
"""
!pip install requests
!pip install pypdf
!pip install python-dotenv
!pip install google-generativeai
"""

In [8]:
# Importing of modules
import requests
from pypdf import PdfReader
from dotenv import load_dotenv
import os
import google.generativeai as genai

In [5]:
load_dotenv()

GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-pro')

In [3]:
# Read from link.txt
link=open("links.txt","r")
data=link.read()
link.close()
print(data)

DEEP NEURAL NETWORKS  @  https://iare-data.s3.ap-south-1.amazonaws.com/uploads/ems/SEE/2023-24/BT7RR20DEC23/13-Dec-2023/ACAC16_qp.pdf
VIRTUAL REALITY  @  https://iare-data.s3.ap-south-1.amazonaws.com/uploads/ems/SEE/2023-24/BT7RR20DEC23/15-Dec-2023/ACDC11_qp.pdf
PREDICTIVE DATA ANALYTICS  @  https://iare-data.s3.ap-south-1.amazonaws.com/uploads/ems/SEE/2023-24/BT7RR20DEC23/18-Dec-2023/ACAC17_qp.pdf
COGNITIVE SCIENCES   @  https://iare-data.s3.ap-south-1.amazonaws.com/uploads/ems/SEE/2023-24/BT7RR20DEC23/20-Dec-2023/ACAC24_qp.pdf
EMBEDDED SYSTEMS  @  https://iare-data.s3.ap-south-1.amazonaws.com/uploads/ems/SEE/2023-24/BT7RR20DEC23/22-Dec-2023/AECC40_qp.pdf



In [4]:
# Convert to dict
subject_links={}
subjects=data.split("\n")

for i in subjects:
    if len(i.strip().split("@"))>1 and i.strip().split("@")[1]!="":
        i=i.strip()
        i_splitted=i.split("@")
        subject_links[i_splitted[0].strip()]=i_splitted[1].strip()

subject_links

{'DEEP NEURAL NETWORKS': 'https://iare-data.s3.ap-south-1.amazonaws.com/uploads/ems/SEE/2023-24/BT7RR20DEC23/13-Dec-2023/ACAC16_qp.pdf',
 'VIRTUAL REALITY': 'https://iare-data.s3.ap-south-1.amazonaws.com/uploads/ems/SEE/2023-24/BT7RR20DEC23/15-Dec-2023/ACDC11_qp.pdf',
 'PREDICTIVE DATA ANALYTICS': 'https://iare-data.s3.ap-south-1.amazonaws.com/uploads/ems/SEE/2023-24/BT7RR20DEC23/18-Dec-2023/ACAC17_qp.pdf',
 'COGNITIVE SCIENCES': 'https://iare-data.s3.ap-south-1.amazonaws.com/uploads/ems/SEE/2023-24/BT7RR20DEC23/20-Dec-2023/ACAC24_qp.pdf',
 'EMBEDDED SYSTEMS': 'https://iare-data.s3.ap-south-1.amazonaws.com/uploads/ems/SEE/2023-24/BT7RR20DEC23/22-Dec-2023/AECC40_qp.pdf'}

In [5]:
# Downlaod the file and save as pdf 
for key,value in subject_links.items():
    url=value
    filename=key.replace("(", "").replace(" ", "_")
    path="pdfs/sem7/"
    extension=".pdf"
    
    try:
        response = requests.get(url)
        response.raise_for_status()
        
        with open(path+filename+extension,"wb") as file:
            file.write(response.content)
        
        print(f"File Downloaded : {filename}")
    except requests.exceptions.RequestException as e:
        print(f"Error Occurred : {filename} {e}")

File Downloaded : DEEP_NEURAL_NETWORKS
File Downloaded : VIRTUAL_REALITY
File Downloaded : PREDICTIVE_DATA_ANALYTICS
File Downloaded : COGNITIVE_SCIENCES
File Downloaded : EMBEDDED_SYSTEMS


In [6]:
# Functions to extract all questions
def clean_question(question):
    question=question[:question.find("[BL: ")].strip()
    question=question.replace("\n", " ")
    if "Page 1 of 2" in question:
        question=question.replace("Page 1 of 2","").strip()
    return question

def get_question_parts(question_group):
    parts = []
    for index, question in enumerate(question_group.strip().split("Marks: 7]")):
        if question != "":
            question = clean_question(question)
            if index % 2 != 0:
                parts.append(parts[index - 1][:2] + " " + question)
            else:
                parts.append(question)
    return parts

def get_questions_in_group(all_text,search_start,search_end):
    specific_module_questions=all_text[all_text.find(search_start)+len(search_start):all_text.find(search_end)]
    return get_question_parts(specific_module_questions)

def extract_all_questions(full_paper):
    all_questions=[]
    actual_questions_container=full_paper[full_paper.find("MODULE – I"):]
    
    all_modules=["MODULE – I","MODULE – II","MODULE – III","MODULE – IV","MODULE – V","− ◦ ◦ ⃝ ◦ ◦ −"]
    
    for i in range(len(all_modules)-1):
        all_questions_in_module_group=get_questions_in_group(actual_questions_container,all_modules[i],all_modules[i+1])
        if len(all_questions_in_module_group)>2:
            all_questions.append(all_questions_in_module_group[:2])
            all_questions.append(all_questions_in_module_group[2:])
        else:
            all_questions.append(all_questions_in_module_group)
    return all_questions

def get_subject_questions(subject_path):
    reader = PdfReader(subject_path)
    full_paper_in_text=""
    for i in range(len(reader.pages)):
        page=reader.pages[i]
        text = page.extract_text()
        full_paper_in_text+=text+"\n"
    all_questions=extract_all_questions(full_paper_in_text)
    return all_questions

def get_all_subject_questions(subject_links):
    subject_data={}
    
    for key in subject_links.keys():
        filename=key.replace("(", "").replace(" ", "_")
        path="pdfs/sem7/"
        extension=".pdf"
        all_questions=get_subject_questions(path+filename+extension)
        subject_data[filename]=all_questions
    
    return subject_data

In [7]:
all_subject_questions=get_all_subject_questions(subject_links)
all_subject_questions

{'DEEP_NEURAL_NETWORKS': [[], [], [], [], []],
 'VIRTUAL_REALITY': [['1. (a) Discuss the role of real-time computer graphics in creating a compelling virtual reality (VR) experience.',
   '1. (b) Analyze the requirements of a virtual environment and their contribution to the effectiveness of VR.'],
  ['2. (a) What is meant by procedural modeling? Discuss the importance of geometrical transformations in creating virtual environments.',
   '2. (b) Explain how collision detection enhances the interactivity of VR systems. Elaborate on 3D boundary representation and its significance in VR.'],
  ['3. (a) Discuss the principles of linear and non-linear interpolation in VR animation.',
   '3. (b) Illustrate with example how free form deformation techniques are used for modeling?'],
  ['4. (a) How elastic collisions are achieved by direct impact of two particles and oblique impact of two particles.',
   '4. (b) Assess the following i) Shape inbetweening ii) Object inbetweening iii) Parametric l

In [ ]:
for subject_name, all_questions in all_subject_questions.items():
    print(f"\n################## {subject_name} ##################\n")

    path = "answers/sem7/"
    extension = ".md"

    with open(path + subject_name + extension, 'w', encoding='utf-8') as file:

        file.write("# " + subject_name + "\n\n")
        for questions_group in all_questions:
            for question in questions_group:
                question_text = question.strip()
                if question_text:
                    print(f" {question_text} ")
                    try:
                        response = model.generate_content(
                            f"""I am currently writing a semester exam for the subject {key}. 
                            The question carries 7 marks, so I need a highly detailed and well-structured answer. 
                            Please provide an elaborate explanation for the following question: '{question_text}'. 
                            The answer should be extremely substantial, clear, and organized in paragraphs and bullet points where necessary to ensure it meets the 7-mark criteria."""
                        )
                        file.write(f"## {question_text}\n\n")
                        file.write(response.text + "\n\n")
                        print(f"[SUCCESS]")
                    except Exception as e:
                        print(f"[ERROR] - {e}")        